<h2>Variables to consider for building a Swachh Bharat dashboard to measure progress under the initiative.</h2>
<p>
For this problem, I parsed xml feed from two different data sources, one for <a href='http://tsc.gov.in/tsc/NDSAP/StatewiseDistrictwisePhysicalProgress.xml'>physical </a> progress and one for <a href='http://tsc.gov.in/tsc/NDSAP/StatewiseDistrictwiseFinancialProgress.xml'>financial </a> progress and finally merged them to get a single big data frame containing both physical and financial progress for all the states and their respective district
</p>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import urllib2
from xml.dom import minidom
%matplotlib inline

In [2]:
def getXML(url):
    """
    Reads in content from the url
    """
    conn = urllib2.urlopen(url)
    return conn.read()

In [92]:
xml_data = getXML('http://tsc.gov.in/tsc/NDSAP/StatewiseDistrictwisePhysicalProgress.xml')

In [93]:
def parseXML(xml_data):
    """
    Parses and returns xml data
    """
    return minidom.parseString(xml_data)

In [94]:
parsed_data = parseXML(xml_data)

In [95]:
rows = parsed_data.getElementsByTagName('row')

In [116]:
def prepareDataForPhysical(rows):
    """
    Takes in rows of parsed xml data
    and returns a pandas data frame
    """
    column_dict = {}
    states = []
    district = []
    obj_ihhl_total = []
    perf_ihhl_total = []
    obj_school_toilets = []
    perf_school_toilets = []
    
    for row in rows:
        states.append(row.getElementsByTagName('State_Name')[0].childNodes[0].nodeValue)
        district.append(row.getElementsByTagName('District_Name')[0].childNodes[0].nodeValue)
        
        obj_ihhl_total.append(float(row.getElementsByTagName('Project_Objectives_IHHL_TOTAL')[0].childNodes[0].nodeValue))
        perf_ihhl_total.append(float(row.getElementsByTagName('Project_Performance-IHHL_TOTAL')[0].childNodes[0].nodeValue))
        obj_school_toilets.append(float(row.getElementsByTagName('Project_Objectives_School_Toilets')[0].childNodes[0].nodeValue))
        perf_school_toilets.append(float(row.getElementsByTagName('Project_Performance-School_Toilets')[0].childNodes[0].nodeValue))
        
    column_dict['states'] = states
    column_dict['district'] = district
    column_dict['obj_ihhl_total'] = obj_ihhl_total
    column_dict['perf_ihhl_total'] = perf_ihhl_total
    column_dict['obj_school_toilets'] = obj_school_toilets
    column_dict['perf_school_toilets'] = perf_school_toilets
      
    return pd.DataFrame(column_dict)

In [117]:
sbm_physical_data = prepareDataForPhysical(rows)

In [118]:
sbm_physical_data.head()

,district,obj_ihhl_total,obj_school_toilets,perf_ihhl_total,perf_school_toilets,states
0,ADILABAD,395656,4462,228731,4462,Andhra Pradesh
1,ANANTAPUR,544649,3421,408557,4258,Andhra Pradesh
2,CHITTOOR,533451,8171,460655,8171,Andhra Pradesh
3,CUDDAPAH,503263,6802,365273,5431,Andhra Pradesh
4,EAST GODAVARI,561655,7004,538705,7004,Andhra Pradesh


In [119]:
xml_data = getXML('http://tsc.gov.in/tsc/NDSAP/StatewiseDistrictwiseFinancialProgress.xml')

In [120]:
parsed_data = parseXML(xml_data)

In [121]:
def prepareDataForFinancial(rows):
    """
    Takes in rows of parsed xml data
    and returns a pandas data frame
    """
    column_dict = {}
    states = [] 
    district = [] 
    total_release_of_funds = []
    total_expenditure_of_funds = []
    
    for row in rows:
        states.append(row.getElementsByTagName('State_Name')[0].childNodes[0].nodeValue)
        district.append(row.getElementsByTagName('District_Name')[0].childNodes[0].nodeValue)
        
        total_release_of_funds.append(float(row.getElementsByTagName('Total_Release_of_funds')[0].childNodes[0].nodeValue))
        total_expenditure_of_funds.append(float(row.getElementsByTagName('Total_Expenditure_Reported')[0].childNodes[0].nodeValue))
        
    column_dict['states'] = states
    column_dict['district'] = district
    column_dict['total_release_of_funds'] = total_release_of_funds
    column_dict['total_expenditure_of_funds'] = total_expenditure_of_funds
       
    return pd.DataFrame(column_dict)

In [122]:
rows = parsed_data.getElementsByTagName('row')

In [123]:
sbm_financial_data = prepareDataForFinancial(rows)

In [124]:
sbm_financial_data.head()

,district,states,total_expenditure_of_funds,total_release_of_funds
0,ADILABAD,ANDHRA PRADESH,4311.34,4993.35
1,ANANTAPUR,ANDHRA PRADESH,4024.28,5001.83
2,CHITTOOR,ANDHRA PRADESH,4002.32,5771.78
3,CUDDAPAH,ANDHRA PRADESH,3829.79,5452.91
4,EAST GODAVARI,ANDHRA PRADESH,7310.09,8270.70


In [130]:
sbm_physical_data.shape

(607, 6)

In [131]:
sbm_financial_data.shape

(607, 4)

In [145]:
sbm_physical_financial = pd.merge(sbm_financial_data, sbm_physical_data, on='district')
sbm_physical_financial.drop('states_y', axis = 1, inplace=True)

In [146]:
sbm_physical_financial.head()

,district,states_x,total_expenditure_of_funds,total_release_of_funds,obj_ihhl_total,obj_school_toilets,perf_ihhl_total,perf_school_toilets
0,ADILABAD,ANDHRA PRADESH,4311.34,4993.35,395656,4462,228731,4462
1,ANANTAPUR,ANDHRA PRADESH,4024.28,5001.83,544649,3421,408557,4258
2,CHITTOOR,ANDHRA PRADESH,4002.32,5771.78,533451,8171,460655,8171
3,CUDDAPAH,ANDHRA PRADESH,3829.79,5452.91,503263,6802,365273,5431
4,EAST GODAVARI,ANDHRA PRADESH,7310.09,8270.70,561655,7004,538705,7004
